In [1]:
#import relevent libraries
import pandas as pd


In [ ]:
#load data into dataframe

file = "allegations_202007271729.csv"

complaints_df = pd.read_csv(file)



In [182]:
# Functions for EDA, cleaning and for later QA

def get_info(dataframe):
    return dataframe.info()

def data_shape(dataframe):
    if dataframe.shape() != (0,0):
        return dataframe.shape()
    else:
        return "There is an error with your dataframe"
    
def dataframe_preview(dataframe):
    return dataframe.head()

def describe_df(dataframe):
    return dataframe.describe()

def unique_counts(dataframe):
    return dataframe.apply(pd.Series.value_counts)

def dataframe_correlation(dataframe):
    return dataframe.corr()

def check_dups(dataframe):
    duplicates = dataframe[dataframe.duplicated()]
    num_dups = len(duplicates)
    if num_dups > 0:
        return f"There are {num_dups} duplicates in your dataset", duplicates
    else:
        return f"There are {num_dups} duplicates in your dataset"

def check_nulls(dataframe):
    num_nulls = dataframe.isnull().sum()
    return num_nulls

def drop_dups(dataframe):
    dataframe = dataframe.drop_duplicates()
    return dataframe

def drop_nulls(dataframe):
    dataframe = dataframe.dropna()
    return dataframe

def change_data_type(dataframe, column_names, data_type):
    dataframe[column_names].astype(data_type)
    return dataframe

#function to combine month and year
def make_complaint_date(dataframe, year_column, month_column):
    dataframe['complaint_date'] = pd.to_datetime(dataframe[year_column].astype(str) + '-' + dataframe[month_column].astype(str) + '-01')
    return dataframe

def make_resolved_date(dataframe, year_column, month_column):
    dataframe['resolved_date'] = pd.to_datetime(dataframe[year_column].astype(str) + '-' + dataframe[month_column].astype(str) + '-01')
    return dataframe
    
#Change M and F to Male and Female respectively
def longform_sex(dataframe, gender_column):
    dataframe[gender_column] = dataframe[gender_column].replace('M', 'Male').replace('F', 'Female')
    return dataframe

#create dictionaries
def create_dictionary(dataframe, first_column, second_column):
   new_dict = dict(zip(dataframe[first_column], dataframe[second_column]))
   return new_dict

#replace abbreviations
def replace_abbreviations(dataframe, dict_name):
    return dataframe.replace(dict_name)
    



EDA Findings:
27 columns with 33,357 rows
Appears to be missing data in some rows
Dates need to be made, currently separate columns for month and day
unique_mos_id, shield_no, complaint_id, precinct are ints and would be better in string format
1985 to 1998 had a null rate of 92% or higher for complainant age, gender, and ethnicity
only 4 rows for the year 2020
noticed discrepancy between gender classification for police and complainants - changed to full form

In [ ]:
get_info(complaints_df)
#code below is to see "missing" columns
#get_info(complaints_df.iloc[:,20:27])

In [ ]:
#get preview of columns, abbreviations for commands and ranks need to be converted to full form
dataframe_preview(complaints_df)

In [ ]:
#unique_mod_id, shield_no, complaint_id, precinct are all ints and would be better as strings, same with year and month columns
#complainant age has a min age of -4301, further analysis will be required
#lots of missing values in complainant age
describe_df(complaints_df)

In [ ]:
#check duplicates
#631 duplicates in the data
check_dups(complaints_df)

In [ ]:
#check nulls
#lots of nulls in command at incident, complainant ethnicity, age, and gender these are the important ones
check_nulls(complaints_df)

In [ ]:
#check rate of nulls by year for complainant ethnicity, age, and gender
#check nulls by year
by_year = complaints_df.groupby('year_received')
null_counts = by_year[['complainant_age_incident', 'complainant_ethnicity', 'complainant_gender']].apply(lambda x: x.isnull().sum())
#null_counts

null_counts['total rows'] = by_year['unique_mos_id'].count()

null_counts['percent of total age'] = null_counts['complainant_age_incident']/null_counts['total rows']
null_counts['percent of total ethnicity'] = null_counts['complainant_ethnicity']/null_counts['total rows']
null_counts['percent of total gender'] = null_counts['complainant_gender']/null_counts['total rows']
null_percentages = null_counts[['percent of total age', 'percent of total ethnicity', 'percent of total gender', 'total rows']]
null_percentages 

In [ ]:
#count of rows by age. 
#Remove ages 9 and under as rows limited.
count_age = complaints_df.groupby('complainant_age_incident').count()
count_age.head(20)

Data Cleaning

In [186]:
#drop duplicates
semi_clean_df = drop_dups(complaints_df)


In [ ]:
#Load abbreviation file and get sheet names
abbreviation_file = 'CCRB Data Layout Table.xlsx'
data_df = pd.ExcelFile(abbreviation_file)
data_df.sheet_names

# read sheets
rank_abbv = pd.read_excel(data_df, 'Rank Abbrevs')
command_abbv = pd.read_excel(data_df, 'Command Abbrevs')

# make dictionaries
ranks = create_dictionary(rank_abbv, 'Abbreviation', 'Rank')
commands = create_dictionary(command_abbv, 'Abbreviation', 'Command Name')


In [190]:
#filter age group
semi_clean_df = semi_clean_df[semi_clean_df['complainant_age_incident'] >= 10]

#replace abbreviations
semi_clean_df = replace_abbreviations(semi_clean_df, ranks)
semi_clean_df = replace_abbreviations(semi_clean_df, commands)

#make dates
semi_clean_df = make_complaint_date(semi_clean_df, 'year_received', 'month_received')
semi_clean_df = make_resolved_date(semi_clean_df, 'year_closed', 'month_closed')

#standardize genders
semi_clean_df = longform_sex(semi_clean_df, 'mos_gender')

#change data types
semi_clean_df = change_data_type(semi_clean_df, column_names=['unique_mos_id', 'shield_no', 'complaint_id', 'precinct'], data_type=str)


#Drop null for semi_clean_data for inital Tableau experiments
semi_clean_drop_null = drop_nulls(semi_clean_df)

#replace nulls with "no record"
Final_df = semi_clean_df.fillna("No Records Available")


In [191]:
#save final_df as csv
Final_df.to_csv('final_df.csv', index=False)

In [193]:
#save semi clean data as csv
semi_clean_drop_null.to_csv('semi_clean_data.csv', index=False)

In [ ]:
#figure out what to do with nulls
#command at incidence 1544 - maybe do count of values in non-null, count $of total then assign at random for nulls
#complainant race, gener, age have most nulls and crucial data
#allegation has 1
#precent has 24
#contact_reason 199
#outcome description 56

so for nulls, put them all as "no record" then on dashboard have a filter drop 2020 because only 4 rows, use 1999 to 2019, because 1985 to 98 had 99% null rate talk about history of ccrb from their website

,percent of total age,percent of total ethnicity,percent of total gender,total rows
year_received,,,,
1985,1.000000,1.000000,1.000000,7
1986,1.000000,1.000000,1.000000,22
1987,1.000000,1.000000,1.000000,22
1988,1.000000,1.000000,1.000000,35
1989,1.000000,1.000000,1.000000,49
1990,1.000000,1.000000,1.000000,54
1991,1.000000,1.000000,1.000000,56
1992,1.000000,1.000000,1.000000,88
1993,1.000000,1.000000,1.000000,76


so for nulls, put them all as "no record" then on dashboard have a filter drop 2020 because only 4 rows, use 1999 to 2019, because 1985 to 98 had 99% null rate talk about history of ccrb from their website

In [ ]:
#updating semi_clean_data to include what is in markdown cell

#filter years want 1999 to 2019

#develop functions to change data type for unique mod id, shield number, complaint id, precinct to strings